In [ ]:
## Following Python Code is used to generate a valid dataframe which could be directly acted upon by STATA,
## from raw data extracted from the CMIE Prowess Database.

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from urllib.request import urlretrieve
%matplotlib inline

In [ ]:
df = pd.read_csv ('.\Data\DF1.csv')


In [ ]:
df.isnull().sum(axis = 0)

Company Name                                                0
NSE symbol                                                  0
Net cash inflow or (outflow) from investing activities      9
Cash (outflow) due to purchase of fixed assets             20
Net fixed assets                                            0
                                                         ... 
Total capital.15                                            0
Deferred tax assets.15                                     36
Cash (outflow) due to dividend paid.15                    138
Profit after tax.15                                         0
Current liabilities & provisions.15                         0
Length: 386, dtype: int64

In [ ]:
basedf = df.iloc[:,0:2]

In [ ]:
dict = {}
for x in range(0,16):
    yearcol = []

    for i in range(349):
        yearcol.append(x+2006)

    dict[x] = df.iloc[:,2+x*24:26+x*24]
    dict[x].insert(0, column='Year', value=yearcol)
    if x==0:
        dict[x]['SPC'] = dict[x]['Total assets']
    else:
        dict[x]['SPC'] = dict[0]['SPC']

    dict[x] = pd.concat([basedf,dict[x]],axis=1)



In [ ]:
columnnames = dict[0].columns
columnnames

Index(['Company Name', 'NSE symbol', 'Year',
       'Net cash inflow or (outflow) from investing activities',
       'Cash (outflow) due to purchase of fixed assets', 'Net fixed assets',
       'Depreciation / Amortisation (net of transfer from revaluation reserves)',
       'Amortisation', 'Net cash flow from operating activities',
       'Total assets', 'Cash balance', 'Bank balance (short term)',
       'Long term borrowings excl current portion', 'Short-term borrowings',
       'Current maturities of long term debt & lease',
       'Short term trade payables and acceptances',
       'Net profit before tax and extra ordinary items', 'Sales',
       'Total income', 'Prior period and extra-ordinary income', 'Net worth',
       'Paid up equity capital (net of forfeited equity capital)',
       'Total capital', 'Deferred tax assets',
       'Cash (outflow) due to dividend paid', 'Profit after tax',
       'Current liabilities & provisions', 'SPC'],
      dtype='object')

In [ ]:
for x in range(0,16):
    dict[x].columns = columnnames

In [ ]:
result = pd.concat([dict[x] for x in dict], axis=0)

In [ ]:
final_df = result.sort_values(by=['Company Name','Year'], ascending=True)

In [ ]:
final_df.isnull().sum(axis = 0)

Company Name                                                                  0
NSE symbol                                                                    0
Year                                                                          0
Net cash inflow or (outflow) from investing activities                       79
Cash (outflow) due to purchase of fixed assets                              205
Net fixed assets                                                              0
Depreciation / Amortisation (net of transfer from revaluation reserves)       1
Amortisation                                                               4568
Net cash flow from operating activities                                      70
Total assets                                                                  0
Cash balance                                                                257
Bank balance (short term)                                                  1778
Long term borrowings excl current portio

In [ ]:
final_df['Depreciation / Amortisation (net of transfer from revaluation reserves)'].head(10)

0     260.5
0     168.8
0     244.0
0     700.6
0     814.1
0    1005.0
0    1091.7
0    2308.2
0    2564.2
0    2291.0
Name: Depreciation / Amortisation (net of transfer from revaluation reserves), dtype: float64

In [ ]:
#CapEX
#Capital already sorted
final_df['CapEx'] = final_df['Cash (outflow) due to purchase of fixed assets']
final_df['CapEx'] = final_df['CapEx'].fillna(final_df.groupby('Company Name')['CapEx'].transform('mean'))
final_df['CapEx'] =  final_df['CapEx']*(-1)
final_df['CapEx'] = np.where(final_df['CapEx'] > 0, final_df['CapEx'], 0 )


In [ ]:
#FCF
final_df['FCF'] = final_df['Net cash flow from operating activities'] - final_df['CapEx']
final_df['FCF'] = final_df['FCF'].fillna(final_df.groupby('Company Name')['FCF'].transform('mean'))
final_df['FCF_1'] = final_df.groupby(['Company Name'])['FCF'].shift(-1)
final_df['FCF_2'] = final_df.groupby(['Company Name'])['FCF'].shift(-2)
final_df['FCF_1'] = final_df['FCF_1']/final_df['Total assets']
final_df['FCF_2'] = final_df['FCF_2']/final_df['Total assets']

In [ ]:
#Leverage
final_df['Leverage'] = final_df['Long term borrowings excl current portion'] + final_df['Short-term borrowings'] + final_df['Current maturities of long term debt & lease']
final_df.Leverage.fillna(final_df['Total assets']-final_df['Net worth'],inplace=True)
final_df['Leverage'] = final_df['Leverage']/final_df['Total assets']

In [ ]:
#Sales
final_df['Revenue'] = final_df['Sales']
final_df.Revenue.fillna(final_df['Total income'],inplace=True)
final_df['Revenue'] = final_df['Revenue']/final_df['Total assets']

In [ ]:
#Cash
final_df['Cash'] = final_df['Cash balance'] + final_df['Bank balance (short term)']
final_df.Cash.fillna(final_df['Cash balance'],inplace=True)
final_df['Cash'] = final_df['Cash'].fillna(final_df.groupby('Company Name')['Cash'].transform('mean'))
final_df['Cash'] = final_df['Cash']/final_df['Total assets']

In [ ]:
#TA
final_df['TA'] = final_df['Total assets']

In [ ]:
#Book Value of Debt for Tobin's q
final_df['BVD'] = final_df['Long term borrowings excl current portion'] + final_df['Short term trade payables and acceptances'] + final_df['Current maturities of long term debt & lease']
final_df.BVD.fillna(final_df['Total capital']-final_df['Paid up equity capital (net of forfeited equity capital)'],inplace=True)

In [ ]:
#Long Term Debt for Firm Leverage Ratio
final_df['LTD'] = final_df['Long term borrowings excl current portion']
final_df.LTD.fillna(final_df['Total assets']-final_df['Net worth'],inplace=True)
final_df['FLR'] = final_df['LTD']/final_df['TA']

In [ ]:
final_df[['Company Name','Year','CapEx','Cash (outflow) due to purchase of fixed assets','Net fixed assets','Depreciation / Amortisation (net of transfer from revaluation reserves)']].head(30)

,Company Name,Year,CapEx,Cash (outflow) due to purchase of fixed assets,Net fixed assets,Depreciation / Amortisation (net of transfer from revaluation reserves)
0,3I Infotech Ltd.,2006,847.800000,-847.8,2889.8,260.5
0,3I Infotech Ltd.,2007,658.000000,-658.0,7153.4,168.8
0,3I Infotech Ltd.,2008,3110.700000,-3110.7,12171.0,244.0
0,3I Infotech Ltd.,2009,4859.800000,-4859.8,22671.2,700.6
0,3I Infotech Ltd.,2010,1698.000000,-1698.0,21848.0,814.1
0,3I Infotech Ltd.,2011,638.000000,-638.0,21751.9,1005.0
0,3I Infotech Ltd.,2012,616.900000,-616.9,24876.5,1091.7
0,3I Infotech Ltd.,2013,425.600000,-425.6,29345.0,2308.2
0,3I Infotech Ltd.,2014,229.100000,-229.1,29460.7,2564.2
0,3I Infotech Ltd.,2015,1023.161538,NaN,19411.7,2291.0


In [ ]:
final_df.isnull().sum(axis = 0)

Company Name                                                                  0
NSE symbol                                                                    0
Year                                                                          0
Net cash inflow or (outflow) from investing activities                       79
Cash (outflow) due to purchase of fixed assets                              205
Net fixed assets                                                              0
Depreciation / Amortisation (net of transfer from revaluation reserves)       1
Amortisation                                                               4568
Net cash flow from operating activities                                      70
Total assets                                                                  0
Cash balance                                                                257
Bank balance (short term)                                                  1778
Long term borrowings excl current portio

In [ ]:
final_df.head(5)

,Company Name,NSE symbol,Year,Net cash inflow or (outflow) from investing activities,Cash (outflow) due to purchase of fixed assets,Net fixed assets,Depreciation / Amortisation (net of transfer from revaluation reserves),Amortisation,Net cash flow from operating activities,Total assets,...,FCF,FCF_1,FCF_2,Leverage,Revenue,Cash,TA,BVD,LTD,FLR
0,3I Infotech Ltd.,3IINFOLTD,2006,-1063.6,-847.8,2889.8,260.5,NaN,263.8,8283.5,...,-584.0,0.061520,-0.024156,0.675789,0.504388,0.023975,8283.5,1006.9,5597.9,0.675789
0,3I Infotech Ltd.,3IINFOLTD,2007,-4447.5,-658.0,7153.4,168.8,NaN,1167.6,13605.1,...,509.6,-0.014708,0.076942,0.707507,0.481672,0.001698,13605.1,1019.0,9625.7,0.707507
0,3I Infotech Ltd.,3IINFOLTD,2008,-6324.7,-3110.7,12171.0,244.0,NaN,2910.6,24209.7,...,-200.1,0.043239,0.071224,0.729745,0.497842,0.069666,24209.7,1102.1,17666.9,0.729745
0,3I Infotech Ltd.,3IINFOLTD,2009,-10411.7,-4859.8,22671.2,700.6,NaN,5906.6,39661.8,...,1046.8,0.043475,0.059342,0.758415,0.576282,0.001962,39661.8,1211.6,30080.1,0.758415
0,3I Infotech Ltd.,3IINFOLTD,2010,-4149.2,-1698.0,21848.0,814.1,NaN,3422.3,39844.7,...,1724.3,0.059069,-0.054248,0.773706,0.614521,0.000095,39844.7,1081.9,30828.1,0.773706


In [ ]:
#Payout Ratio
final_df['Net profit before tax and extra ordinary items'] = final_df['Net profit before tax and extra ordinary items'].fillna(final_df.groupby('Company Name')['Net profit before tax and extra ordinary items'].transform('mean'))
final_df['Cash (outflow) due to dividend paid'] = final_df['Cash (outflow) due to dividend paid'].fillna(0)
final_df['PR'] = ((-1)*final_df['Cash (outflow) due to dividend paid'])/abs(final_df['Net profit before tax and extra ordinary items'])
final_df['PR'] = final_df['PR'].fillna(0)

In [ ]:
final_df.isnull().sum(axis = 0)

Company Name                                                                  0
NSE symbol                                                                    0
Year                                                                          0
Net cash inflow or (outflow) from investing activities                       79
Cash (outflow) due to purchase of fixed assets                              205
Net fixed assets                                                              0
Depreciation / Amortisation (net of transfer from revaluation reserves)       1
Amortisation                                                               4568
Net cash flow from operating activities                                      70
Total assets                                                                  0
Cash balance                                                                257
Bank balance (short term)                                                  1778
Long term borrowings excl current portio

In [ ]:
#KZ Index
final_df['DA'] = final_df['Depreciation / Amortisation (net of transfer from revaluation reserves)'] + final_df['Amortisation']
final_df['DA'] = final_df['DA'].fillna(final_df['Depreciation / Amortisation (net of transfer from revaluation reserves)'])
final_df['CF'] = final_df['Net profit before tax and extra ordinary items'] + final_df['DA']
final_df['CF'] = final_df['CF'].fillna(final_df.groupby('Company Name')['CF'].transform('mean'))


In [ ]:
final_df.isnull().sum(axis = 0)

Company Name                                                                  0
NSE symbol                                                                    0
Year                                                                          0
Net cash inflow or (outflow) from investing activities                       79
Cash (outflow) due to purchase of fixed assets                              205
Net fixed assets                                                              0
Depreciation / Amortisation (net of transfer from revaluation reserves)       1
Amortisation                                                               4568
Net cash flow from operating activities                                      70
Total assets                                                                  0
Cash balance                                                                257
Bank balance (short term)                                                  1778
Long term borrowings excl current portio

In [ ]:
secdf = pd.read_csv (r'D:\Academics\SEM6\AE\Report\Data\DF2.csv')

C:\Users\Madhav\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (1,2,3,4,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,33,34,35,36,41,42,43,44,49,50,51,52,53,54,55,56,57,58,59,60,65,66,67,68,73,74,75,76,77,78,79,80,81,82,83,84,93,94,95,96,97,98,99,100,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,125,126,127,128,133,134,135,136,141,142,143,144,145,146,147,148,153,154,155,156,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,197,198,199,200,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,241,242,243,244,245,246,247,248,249,250,251,252,257,258,259,260,261,262,263,264,277,278,279,280,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,317,318,319,320,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,357,358,359,360,361,362,363,364,365,366,367,368,373,374,375,376,385,386,387,388,3

In [ ]:
secbasedf = secdf[['Date']]

In [ ]:
dict2 = {}
columnnames = ['Date','ACP','SO','MC','ST']
for x in range(0,349):
    dict2[x] = secdf.iloc[:,1+x*4:5+x*4]
    cn = secdf.columns[1+x*4]
    cname = (cn.split("-"))[0]
    cnl = []
    for i in range(3966):
        cnl.append(cname)
    dict2[x] = pd.concat([secbasedf,dict2[x]],axis=1)
    dict2[x].columns = columnnames
    dict2[x].insert(loc=0, column='Company name', value=cnl)
    dict2[x]['Date'] = pd.to_datetime(dict2[x]['Date'])

In [ ]:
for x in range(0,349):
    dict2[x]['Date'] = pd.to_datetime(dict2[x]['Date'],errors='coerce')
    dict2[x]['ACP'] = pd.to_numeric(dict2[x]['ACP'],errors='coerce')
    dict2[x]['SO'] = pd.to_numeric(dict2[x]['SO'],errors='coerce')
    dict2[x]['MC'] = pd.to_numeric(dict2[x]['MC'],errors='coerce')
    dict2[x]['ST'] = pd.to_numeric(dict2[x]['ST'],errors='coerce')

In [ ]:
for x in range(0,349):
    dict2[x]['DR'] = abs(dict2[x]['ACP'].pct_change(1))
    dict2[x]['Amihud'] = (10000)*np.log1p(((dict2[x]['DR'])/(dict2[x]['ACP']*dict2[x]['ST'])))
    dict2[x]['Turnover'] = (1)*np.log1p(((dict2[x]['ST'])/(dict2[x]['SO'])))
    dict2[x]=dict2[x].groupby([dict2[x]['Company name'],dict2[x]['Date'].dt.year])['MC','Amihud','Turnover'].agg(['mean'])
    dict2[x].columns = ['MC','Amihud','Turnover']
    dict2[x].reset_index(inplace=True)
    dict2[x].columns = ['Company Name','Year','MC','Amihud','Turnover']

<ipython-input-1351-c69933e6b57e>:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dict2[x]=dict2[x].groupby([dict2[x]['Company name'],dict2[x]['Date'].dt.year])['MC','Amihud','Turnover'].agg(['mean'])


In [ ]:
secresult = pd.concat([dict2[x] for x in dict2], axis=0)

In [ ]:
final_df

,Company Name,NSE symbol,Year,Net cash inflow or (outflow) from investing activities,Cash (outflow) due to purchase of fixed assets,Net fixed assets,Depreciation / Amortisation (net of transfer from revaluation reserves),Amortisation,Net cash flow from operating activities,Total assets,...,Leverage,Revenue,Cash,TA,BVD,LTD,FLR,PR,DA,CF
0,3I Infotech Ltd.,3IINFOLTD,2006,-1063.6,-847.8,2889.8,260.5,NaN,263.8,8283.5,...,0.675789,0.504388,0.023975,8283.5,1006.9,5597.9,0.675789,0.223391,260.5,840.2
0,3I Infotech Ltd.,3IINFOLTD,2007,-4447.5,-658.0,7153.4,168.8,NaN,1167.6,13605.1,...,0.707507,0.481672,0.001698,13605.1,1019.0,9625.7,0.707507,0.176272,168.8,1267.1
0,3I Infotech Ltd.,3IINFOLTD,2008,-6324.7,-3110.7,12171.0,244.0,NaN,2910.6,24209.7,...,0.729745,0.497842,0.069666,24209.7,1102.1,17666.9,0.729745,0.110691,244.0,2227.0
0,3I Infotech Ltd.,3IINFOLTD,2009,-10411.7,-4859.8,22671.2,700.6,NaN,5906.6,39661.8,...,0.758415,0.576282,0.001962,39661.8,1211.6,30080.1,0.758415,0.105349,700.6,3585.3
0,3I Infotech Ltd.,3IINFOLTD,2010,-4149.2,-1698.0,21848.0,814.1,NaN,3422.3,39844.7,...,0.773706,0.614521,0.000095,39844.7,1081.9,30828.1,0.773706,0.109679,814.1,3583.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,Zodiac Clothing Co. Ltd.,ZODIACLOTH,2017,160.6,-111.9,1342.6,118.0,NaN,29.2,4260.4,...,0.138954,0.671134,0.032650,4260.4,425.7,89.3,0.020960,0.147839,118.0,-13.9
348,Zodiac Clothing Co. Ltd.,ZODIACLOTH,2018,341.7,-32.7,1268.8,108.4,NaN,-365.5,4255.3,...,0.157122,0.567951,0.029981,4255.3,487.3,70.6,0.016591,0.074314,108.4,-154.0
348,Zodiac Clothing Co. Ltd.,ZODIACLOTH,2019,-80.3,-56.8,1207.6,107.1,NaN,-272.7,4254.6,...,0.149626,0.520636,0.029986,4254.6,267.0,8.3,0.001951,0.088717,107.1,-112.7
348,Zodiac Clothing Co. Ltd.,ZODIACLOTH,2020,330.5,-45.6,1857.6,354.6,NaN,107.0,5132.4,...,0.276732,0.384460,0.042923,5132.4,977.1,729.0,0.142039,0.070882,354.6,54.1


In [ ]:
secresult

,Company Name,Year,MC,Amihud,Turnover
0,3I Infotech Ltd.,2006,9185.572560,0.000009,0.006357
1,3I Infotech Ltd.,2007,17065.915663,0.000003,0.009883
2,3I Infotech Ltd.,2008,12601.775732,0.000015,0.002952
3,3I Infotech Ltd.,2009,9415.913498,0.000011,0.015434
4,3I Infotech Ltd.,2010,12737.740476,0.000002,0.007061
...,...,...,...,...,...
11,Zodiac Clothing Co. Ltd.,2017,3992.032389,0.000490,0.000467
12,Zodiac Clothing Co. Ltd.,2018,3253.201463,0.000962,0.000423
13,Zodiac Clothing Co. Ltd.,2019,4351.963102,0.001458,0.000410
14,Zodiac Clothing Co. Ltd.,2020,2740.639921,0.000854,0.000806


In [ ]:
new_df = pd.merge(final_df, secresult,  how='left', left_on=['Company Name','Year'], right_on = ['Company Name','Year'])

In [ ]:
new_df

,Company Name,NSE symbol,Year,Net cash inflow or (outflow) from investing activities,Cash (outflow) due to purchase of fixed assets,Net fixed assets,Depreciation / Amortisation (net of transfer from revaluation reserves),Amortisation,Net cash flow from operating activities,Total assets,...,TA,BVD,LTD,FLR,PR,DA,CF,MC,Amihud,Turnover
0,3I Infotech Ltd.,3IINFOLTD,2006,-1063.6,-847.8,2889.8,260.5,NaN,263.8,8283.5,...,8283.5,1006.9,5597.9,0.675789,0.223391,260.5,840.2,9185.572560,0.000009,0.006357
1,3I Infotech Ltd.,3IINFOLTD,2007,-4447.5,-658.0,7153.4,168.8,NaN,1167.6,13605.1,...,13605.1,1019.0,9625.7,0.707507,0.176272,168.8,1267.1,17065.915663,0.000003,0.009883
2,3I Infotech Ltd.,3IINFOLTD,2008,-6324.7,-3110.7,12171.0,244.0,NaN,2910.6,24209.7,...,24209.7,1102.1,17666.9,0.729745,0.110691,244.0,2227.0,12601.775732,0.000015,0.002952
3,3I Infotech Ltd.,3IINFOLTD,2009,-10411.7,-4859.8,22671.2,700.6,NaN,5906.6,39661.8,...,39661.8,1211.6,30080.1,0.758415,0.105349,700.6,3585.3,9415.913498,0.000011,0.015434
4,3I Infotech Ltd.,3IINFOLTD,2010,-4149.2,-1698.0,21848.0,814.1,NaN,3422.3,39844.7,...,39844.7,1081.9,30828.1,0.773706,0.109679,814.1,3583.1,12737.740476,0.000002,0.007061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5579,Zodiac Clothing Co. Ltd.,ZODIACLOTH,2017,160.6,-111.9,1342.6,118.0,NaN,29.2,4260.4,...,4260.4,425.7,89.3,0.020960,0.147839,118.0,-13.9,3992.032389,0.000490,0.000467
5580,Zodiac Clothing Co. Ltd.,ZODIACLOTH,2018,341.7,-32.7,1268.8,108.4,NaN,-365.5,4255.3,...,4255.3,487.3,70.6,0.016591,0.074314,108.4,-154.0,3253.201463,0.000962,0.000423
5581,Zodiac Clothing Co. Ltd.,ZODIACLOTH,2019,-80.3,-56.8,1207.6,107.1,NaN,-272.7,4254.6,...,4254.6,267.0,8.3,0.001951,0.088717,107.1,-112.7,4351.963102,0.001458,0.000410
5582,Zodiac Clothing Co. Ltd.,ZODIACLOTH,2020,330.5,-45.6,1857.6,354.6,NaN,107.0,5132.4,...,5132.4,977.1,729.0,0.142039,0.070882,354.6,54.1,2740.639921,0.000854,0.000806


In [ ]:
new_df.isnull().sum(axis = 0)

Company Name                                                                  0
NSE symbol                                                                    0
Year                                                                          0
Net cash inflow or (outflow) from investing activities                       79
Cash (outflow) due to purchase of fixed assets                              205
Net fixed assets                                                              0
Depreciation / Amortisation (net of transfer from revaluation reserves)       1
Amortisation                                                               4568
Net cash flow from operating activities                                      70
Total assets                                                                  0
Cash balance                                                                257
Bank balance (short term)                                                  1778
Long term borrowings excl current portio

In [ ]:
new_df['MC'] = new_df['MC'].fillna(new_df.groupby('Company Name')['MC'].transform('mean'))
new_df['Amihud'] = new_df['Amihud'].fillna(new_df.groupby('Company Name')['Amihud'].transform('mean'))
new_df['Turnover'] = new_df['Turnover'].fillna(new_df.groupby('Company Name')['Turnover'].transform('mean'))

In [ ]:
new_df.isnull().sum(axis = 0)

Company Name                                                                  0
NSE symbol                                                                    0
Year                                                                          0
Net cash inflow or (outflow) from investing activities                       79
Cash (outflow) due to purchase of fixed assets                              205
Net fixed assets                                                              0
Depreciation / Amortisation (net of transfer from revaluation reserves)       1
Amortisation                                                               4568
Net cash flow from operating activities                                      70
Total assets                                                                  0
Cash balance                                                                257
Bank balance (short term)                                                  1778
Long term borrowings excl current portio

In [ ]:
new_df = new_df.dropna(subset=['Cash', 'MC','Amihud','Turnover'])

In [ ]:
new_df.isnull().sum(axis = 0)

Company Name                                                                  0
NSE symbol                                                                    0
Year                                                                          0
Net cash inflow or (outflow) from investing activities                       78
Cash (outflow) due to purchase of fixed assets                              204
Net fixed assets                                                              0
Depreciation / Amortisation (net of transfer from revaluation reserves)       1
Amortisation                                                               4503
Net cash flow from operating activities                                      69
Total assets                                                                  0
Cash balance                                                                241
Bank balance (short term)                                                  1753
Long term borrowings excl current portio

In [ ]:
new_df


,Company Name,NSE symbol,Year,Net cash inflow or (outflow) from investing activities,Cash (outflow) due to purchase of fixed assets,Net fixed assets,Depreciation / Amortisation (net of transfer from revaluation reserves),Amortisation,Net cash flow from operating activities,Total assets,...,TA,BVD,LTD,FLR,PR,DA,CF,MC,Amihud,Turnover
0,3I Infotech Ltd.,3IINFOLTD,2006,-1063.6,-847.8,2889.8,260.5,NaN,263.8,8283.5,...,8283.5,1006.9,5597.9,0.675789,0.223391,260.5,840.2,9185.572560,0.000009,0.006357
1,3I Infotech Ltd.,3IINFOLTD,2007,-4447.5,-658.0,7153.4,168.8,NaN,1167.6,13605.1,...,13605.1,1019.0,9625.7,0.707507,0.176272,168.8,1267.1,17065.915663,0.000003,0.009883
2,3I Infotech Ltd.,3IINFOLTD,2008,-6324.7,-3110.7,12171.0,244.0,NaN,2910.6,24209.7,...,24209.7,1102.1,17666.9,0.729745,0.110691,244.0,2227.0,12601.775732,0.000015,0.002952
3,3I Infotech Ltd.,3IINFOLTD,2009,-10411.7,-4859.8,22671.2,700.6,NaN,5906.6,39661.8,...,39661.8,1211.6,30080.1,0.758415,0.105349,700.6,3585.3,9415.913498,0.000011,0.015434
4,3I Infotech Ltd.,3IINFOLTD,2010,-4149.2,-1698.0,21848.0,814.1,NaN,3422.3,39844.7,...,39844.7,1081.9,30828.1,0.773706,0.109679,814.1,3583.1,12737.740476,0.000002,0.007061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5579,Zodiac Clothing Co. Ltd.,ZODIACLOTH,2017,160.6,-111.9,1342.6,118.0,NaN,29.2,4260.4,...,4260.4,425.7,89.3,0.020960,0.147839,118.0,-13.9,3992.032389,0.000490,0.000467
5580,Zodiac Clothing Co. Ltd.,ZODIACLOTH,2018,341.7,-32.7,1268.8,108.4,NaN,-365.5,4255.3,...,4255.3,487.3,70.6,0.016591,0.074314,108.4,-154.0,3253.201463,0.000962,0.000423
5581,Zodiac Clothing Co. Ltd.,ZODIACLOTH,2019,-80.3,-56.8,1207.6,107.1,NaN,-272.7,4254.6,...,4254.6,267.0,8.3,0.001951,0.088717,107.1,-112.7,4351.963102,0.001458,0.000410
5582,Zodiac Clothing Co. Ltd.,ZODIACLOTH,2020,330.5,-45.6,1857.6,354.6,NaN,107.0,5132.4,...,5132.4,977.1,729.0,0.142039,0.070882,354.6,54.1,2740.639921,0.000854,0.000806


In [ ]:
new_df.isnull().sum(axis = 0)

Company Name                                                                  0
NSE symbol                                                                    0
Year                                                                          0
Net cash inflow or (outflow) from investing activities                       78
Cash (outflow) due to purchase of fixed assets                              204
Net fixed assets                                                              0
Depreciation / Amortisation (net of transfer from revaluation reserves)       1
Amortisation                                                               4503
Net cash flow from operating activities                                      69
Total assets                                                                  0
Cash balance                                                                241
Bank balance (short term)                                                  1753
Long term borrowings excl current portio

In [ ]:
#KZ index continue
# K = Net fixed asset
# Debt = BVD
#
#final_df['DA'] = final_df['Depreciation / Amortisation (net of transfer from revaluation reserves)'] + final_df['Amortisation']
#final_df['DA'] = final_df['DA'].fillna(final_df['Depreciation / Amortisation (net of transfer from revaluation reserves)'])
#final_df['CF'] = final_df['Net profit before tax and extra ordinary items'] + final_df['DA']
#final_df['CF'] = final_df['CF'].fillna(final_df.groupby('Company Name')['CF'].transform('mean'))
new_df['Q'] = (new_df['MC']+new_df['Net worth']-new_df['Paid up equity capital (net of forfeited equity capital)'] - new_df['Deferred tax assets'])/(new_df['Net worth'])
new_df['Q'] = new_df['Q'].fillna((new_df['MC']+new_df['Net worth']-new_df['Paid up equity capital (net of forfeited equity capital)'])/(new_df['Net worth']))
new_df['K'] = new_df.groupby(['Company Name'])['Net fixed assets'].shift(1)
new_df['KZ'] =  (-1.001909)*(new_df['CF']/new_df['K']) + (0.2826389)*(new_df['Q']) + (3.139193)*(new_df['BVD']/new_df['Total capital']) + (-39.3678)*(new_df['Cash (outflow) due to dividend paid']/new_df['K']) + (-1.314759)*(new_df['Cash']/new_df['K'])

<ipython-input-1364-8309e5fa3623>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Q'] = (new_df['MC']+new_df['Net worth']-new_df['Paid up equity capital (net of forfeited equity capital)'] - new_df['Deferred tax assets'])/(new_df['Net worth'])
<ipython-input-1364-8309e5fa3623>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Q'] = new_df['Q'].fillna((new_df['MC']+new_df['Net worth']-new_df['Paid up equity capital (net of forfeited equity capital)'])/(new_df['Net worth']))
<ipython-input-13

In [ ]:
#Tobin's Q
new_df['TQ'] = (new_df['MC'] + new_df['BVD']) / (new_df['TA'])
new_df['TQ_diff'] = new_df.groupby(['Company Name'])['TQ'].diff()
new_df['TQ_diff_1'] = new_df.groupby(['Company Name'])['TQ_diff'].shift(1)
new_df['TQ_diff_2'] = new_df.groupby(['Company Name'])['TQ_diff'].shift(2)

<ipython-input-1365-8c98e210e11c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['TQ'] = (new_df['MC'] + new_df['BVD']) / (new_df['TA'])
<ipython-input-1365-8c98e210e11c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['TQ_diff'] = new_df.groupby(['Company Name'])['TQ'].diff()
<ipython-input-1365-8c98e210e11c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

In [ ]:
new_df.isnull().sum(axis = 0)

Company Name                                                                  0
NSE symbol                                                                    0
Year                                                                          0
Net cash inflow or (outflow) from investing activities                       78
Cash (outflow) due to purchase of fixed assets                              204
Net fixed assets                                                              0
Depreciation / Amortisation (net of transfer from revaluation reserves)       1
Amortisation                                                               4503
Net cash flow from operating activities                                      69
Total assets                                                                  0
Cash balance                                                                241
Bank balance (short term)                                                  1753
Long term borrowings excl current portio

In [ ]:
new_df.to_excel("newdf.xlsx")

In [ ]:
reg_table = new_df.iloc[:,28:]
reg_name_table = new_df.iloc[:,0:3]
reg_name_table = pd.concat([reg_name_table,reg_table],axis=1)

In [ ]:
reg_name_table.columns
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
reg_name_table['FirmID'] = le.fit_transform(reg_name_table['NSE symbol'])
reg_name_table['FirmID'] = 1+reg_name_table['FirmID']

In [ ]:
reg_name_table_sec1 = reg_name_table[['FirmID','Company Name', 'NSE symbol', 'Year', 'CapEx','TA','FCF','FCF_1','FCF_2','Leverage',
       'Revenue', 'Cash','TQ','TQ_diff_1','TQ_diff_2','Amihud', 'Turnover','FLR','PR','KZ']]


In [ ]:
#Section1
reg_name_table_sec1

,FirmID,Company Name,NSE symbol,Year,CapEx,TA,FCF,FCF_1,FCF_2,Leverage,Revenue,Cash,TQ,TQ_diff_1,TQ_diff_2,Amihud,Turnover,FLR,PR,KZ
0,2,3I Infotech Ltd.,3IINFOLTD,2006,847.8,8283.5,-584.0,0.061520,-0.024156,0.675789,0.504388,0.023975,1.230455,NaN,NaN,0.000009,0.006357,0.675789,0.223391,NaN
1,2,3I Infotech Ltd.,3IINFOLTD,2007,658.0,13605.1,509.6,-0.014708,0.076942,0.707507,0.481672,0.001698,1.329275,NaN,NaN,0.000003,0.009883,0.707507,0.176272,5.646263
2,2,3I Infotech Ltd.,3IINFOLTD,2008,3110.7,24209.7,-200.1,0.043239,0.071224,0.729745,0.497842,0.069666,0.566049,0.098820,NaN,0.000015,0.002952,0.729745,0.110691,3.080205
3,2,3I Infotech Ltd.,3IINFOLTD,2009,4859.8,39661.8,1046.8,0.043475,0.059342,0.758415,0.576282,0.001962,0.267953,-0.763226,0.098820,0.000011,0.015434,0.758415,0.105349,2.695169
4,2,3I Infotech Ltd.,3IINFOLTD,2010,1698.0,39844.7,1724.3,0.059069,-0.054248,0.773706,0.614521,0.000095,0.346838,-0.298096,-0.763226,0.000002,0.007061,0.773706,0.109679,2.189047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5579,343,Zodiac Clothing Co. Ltd.,ZODIACLOTH,2017,111.9,4260.4,-82.7,-0.093465,-0.077340,0.138954,0.671134,0.032650,1.036929,-0.396002,-0.211971,0.000490,0.000467,0.020960,0.147839,8.122132
5580,343,Zodiac Clothing Co. Ltd.,ZODIACLOTH,2018,32.7,4255.3,-398.2,-0.077433,0.014429,0.157122,0.567951,0.029981,0.879022,-0.114735,-0.396002,0.000962,0.000423,0.016591,0.074314,9.153146
5581,343,Zodiac Clothing Co. Ltd.,ZODIACLOTH,2019,56.8,4254.6,-329.5,0.014431,-0.023716,0.149626,0.520636,0.029986,1.085640,-0.157907,-0.114735,0.001458,0.000410,0.001951,0.088717,3.715685
5582,343,Zodiac Clothing Co. Ltd.,ZODIACLOTH,2020,45.6,5132.4,61.4,-0.019659,NaN,0.276732,0.384460,0.042923,0.724367,0.206618,-0.157907,0.000854,0.000806,0.142039,0.070882,14.832684


In [ ]:
reg_name_table_sec1['I_1'] = reg_name_table_sec1.groupby(['Company Name'])['CapEx'].shift(-1)
reg_name_table_sec1['I_2'] = reg_name_table_sec1.groupby(['Company Name'])['CapEx'].shift(-2)
reg_name_table_sec1['I/TA_1'] = reg_name_table_sec1['I_1']/reg_name_table_sec1['TA']
reg_name_table_sec1['I/TA_2'] = reg_name_table_sec1['I_2']/reg_name_table_sec1['TA']

<ipython-input-1373-99d3574988a6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reg_name_table_sec1['I_1'] = reg_name_table_sec1.groupby(['Company Name'])['CapEx'].shift(-1)
<ipython-input-1373-99d3574988a6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reg_name_table_sec1['I_2'] = reg_name_table_sec1.groupby(['Company Name'])['CapEx'].shift(-2)
<ipython-input-1373-99d3574988a6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [ ]:
reg_name_table_sec1 = reg_name_table_sec1.sort_values(by=['FirmID','Year'], ascending=True)
reg_name_table_sec1.head(16)

,FirmID,Company Name,NSE symbol,Year,CapEx,TA,FCF,FCF_1,FCF_2,Leverage,...,TQ_diff_2,Amihud,Turnover,FLR,PR,KZ,I_1,I_2,I/TA_1,I/TA_2
5056,1,Twentyfirst Century Mgmt. Services Ltd.,21STCENMGM,2006,13.30,405.1,99.10,-0.026660,-0.133547,0.315971,...,NaN,0.079698,0.000535,0.315971,-0.000000,NaN,5.60,7.30,0.013824,0.018020
5057,1,Twentyfirst Century Mgmt. Services Ltd.,21STCENMGM,2007,5.60,810.3,-10.80,-0.066765,-0.367025,0.329878,...,NaN,0.079698,0.000535,0.329878,-0.000000,-19.826341,7.30,0.70,0.009009,0.000864
5058,1,Twentyfirst Century Mgmt. Services Ltd.,21STCENMGM,2008,7.30,1256.6,-54.10,-0.236670,0.126452,0.393283,...,NaN,0.079698,0.000535,0.393283,0.078985,25.831194,0.70,2.80,0.000557,0.002228
5059,1,Twentyfirst Century Mgmt. Services Ltd.,21STCENMGM,2009,0.70,775.8,-297.40,0.204821,0.023460,0.338360,...,-0.319140,0.079698,0.000535,0.338360,0.038153,24.631360,2.80,18.90,0.003609,0.024362
5060,1,Twentyfirst Century Mgmt. Services Ltd.,21STCENMGM,2010,2.80,887.8,158.90,0.020500,0.035594,0.440077,...,-0.104028,0.079698,0.000535,0.440077,-0.000000,2.361443,18.90,1.20,0.021289,0.001352
5061,1,Twentyfirst Century Mgmt. Services Ltd.,21STCENMGM,2011,18.90,915.1,18.20,0.034532,0.004098,0.403890,...,0.194393,0.079698,0.000535,0.403890,-0.000000,-1.608839,1.20,5.85,0.001311,0.006393
5062,1,Twentyfirst Century Mgmt. Services Ltd.,21STCENMGM,2012,1.20,799.3,31.60,0.004692,-0.154635,0.455023,...,-0.031455,0.079698,0.000535,0.455023,-0.000000,3.769363,5.85,0.00,0.007319,0.000000
5063,1,Twentyfirst Century Mgmt. Services Ltd.,21STCENMGM,2013,5.85,691.0,3.75,-0.178871,0.368596,0.424023,...,-0.010496,0.079698,0.000535,0.424023,-0.000000,2.749510,0.00,18.80,0.000000,0.027207
5064,1,Twentyfirst Century Mgmt. Services Ltd.,21STCENMGM,2014,0.00,549.7,-123.60,0.463344,-0.170820,0.287611,...,0.049449,0.079698,0.000535,0.287611,-0.000000,1.785128,18.80,0.20,0.034200,0.000364
5065,1,Twentyfirst Century Mgmt. Services Ltd.,21STCENMGM,2015,18.80,946.1,254.70,-0.099250,0.095392,0.423317,...,0.061244,0.079698,0.000535,0.423317,-0.000000,-12.795118,0.20,5.85,0.000211,0.006183


In [ ]:
exclude1 = list(reg_name_table_sec1[reg_name_table_sec1['I/TA_1']>1]['Company Name'])
exclude2 = list(reg_name_table_sec1[reg_name_table_sec1['I/TA_2']>1]['Company Name'])

In [ ]:
exclude = list(set(exclude1 + exclude2))

In [ ]:
# Removing firms with I_TA_1 or I_TA_2 greater than 1. Thus we can apply logistic transformation on the dependent variable
# whilst keeping the panel data balanced.
reg_name_table_sec1 = reg_name_table_sec1[~reg_name_table_sec1['Company Name'].isin(exclude)]

In [ ]:
#Log Transformations
reg_name_table_sec1['LT(I/TA_1)'] = np.log(reg_name_table_sec1['I/TA_1']/(1-reg_name_table_sec1['I/TA_1']))
reg_name_table_sec1['LT(I/TA_2)'] = np.log(reg_name_table_sec1['I/TA_2']/(1-reg_name_table_sec1['I/TA_2']))
reg_name_table_sec1['LT(FCF_1)'] = np.log1p(reg_name_table_sec1['FCF_1'])
reg_name_table_sec1['LT(FCF_2)'] = np.log1p(reg_name_table_sec1['FCF_2'])
reg_name_table_sec1['LT(Leverage)'] = np.log1p(reg_name_table_sec1['Leverage'])
reg_name_table_sec1['LT(Revenue)'] = np.log1p(reg_name_table_sec1['Revenue'])
reg_name_table_sec1['LT(Cash)'] = np.log1p(reg_name_table_sec1['Cash'])
reg_name_table_sec1['LT(TQ)'] = np.log1p(reg_name_table_sec1['TQ'])
reg_name_table_sec1['LT(TQ_diff_1)'] = np.log1p(reg_name_table_sec1['TQ_diff_1'])
reg_name_table_sec1['LT(TQ_diff_2)'] = np.log1p(reg_name_table_sec1['TQ_diff_2'])

C:\Users\Madhav\anaconda3\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Madhav\anaconda3\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
#Fixing Infinity Values
reg_name_table_sec1 = reg_name_table_sec1.replace([np.inf, -np.inf], np.nan)
reg_name_table_sec1['LT(I/TA_1)'] = reg_name_table_sec1['LT(I/TA_1)'].fillna(reg_name_table_sec1.groupby('Company Name')['LT(I/TA_1)'].transform('max'))
reg_name_table_sec1['LT(I/TA_2)'] = reg_name_table_sec1['LT(I/TA_2)'].fillna(reg_name_table_sec1.groupby('Company Name')['LT(I/TA_2)'].transform('max'))

In [ ]:
reg_name_table_sec1.isnull().sum(axis = 0)

FirmID              0
Company Name        0
NSE symbol          0
Year                0
CapEx               0
TA                  0
FCF                 0
FCF_1             327
FCF_2             654
Leverage            0
Revenue             0
Cash                0
TQ                  0
TQ_diff_1         654
TQ_diff_2         981
Amihud              0
Turnover            0
FLR                 0
PR                  0
KZ                327
I_1               327
I_2               654
I/TA_1            327
I/TA_2            654
LT(I/TA_1)          0
LT(I/TA_2)          0
LT(FCF_1)         330
LT(FCF_2)         660
LT(Leverage)        0
LT(Revenue)         0
LT(Cash)            0
LT(TQ)              0
LT(TQ_diff_1)     937
LT(TQ_diff_2)    1253
dtype: int64

In [ ]:
reg_name_table_sec1.head(12)

,FirmID,Company Name,NSE symbol,Year,CapEx,TA,FCF,FCF_1,FCF_2,Leverage,...,LT(I/TA_1),LT(I/TA_2),LT(FCF_1),LT(FCF_2),LT(Leverage),LT(Revenue),LT(Cash),LT(TQ),LT(TQ_diff_1),LT(TQ_diff_2)
5056,1,Twentyfirst Century Mgmt. Services Ltd.,21STCENMGM,2006,13.30,405.1,99.10,-0.026660,-0.133547,0.315971,...,-4.267447,-3.998075,-0.027022,-0.143348,0.274575,0.477137,0.001480,0.477502,NaN,NaN
5057,1,Twentyfirst Century Mgmt. Services Ltd.,21STCENMGM,2007,5.60,810.3,-10.80,-0.066765,-0.367025,0.329878,...,-4.700480,-7.053215,-0.069099,-0.457324,0.285087,0.399731,0.001603,0.256889,NaN,NaN
5058,1,Twentyfirst Century Mgmt. Services Ltd.,21STCENMGM,2008,7.30,1256.6,-54.10,-0.236670,0.126452,0.393283,...,-7.492283,-6.104315,-0.270065,0.119073,0.331663,0.262315,0.001113,0.173006,-0.384399,NaN
5059,1,Twentyfirst Century Mgmt. Services Ltd.,21STCENMGM,2009,0.70,775.8,-297.40,0.204821,0.023460,0.338360,...,-5.620660,-3.690069,0.186331,0.023189,0.291445,0.068374,0.002189,0.324448,-0.109847,-0.384399
5060,1,Twentyfirst Century Mgmt. Services Ltd.,21STCENMGM,2010,2.80,887.8,158.90,0.020500,0.035594,0.440077,...,-3.828066,-6.605072,0.020293,0.034975,0.364696,0.068873,0.001913,0.301446,0.177638,-0.109847
5061,1,Twentyfirst Century Mgmt. Services Ltd.,21STCENMGM,2011,18.90,915.1,18.20,0.034532,0.004098,0.403890,...,-6.635400,-5.046178,0.033949,0.004090,0.339247,0.285932,0.166053,0.293651,-0.031960,0.177638
5062,1,Twentyfirst Century Mgmt. Services Ltd.,21STCENMGM,2012,1.20,799.3,31.60,0.004692,-0.154635,0.455023,...,-4.909949,-3.576699,0.004681,-0.167987,0.375022,0.083031,0.112752,0.329854,-0.010551,-0.031960
5063,1,Twentyfirst Century Mgmt. Services Ltd.,21STCENMGM,2013,5.85,691.0,3.75,-0.178871,0.368596,0.424023,...,-3.340717,-3.576699,-0.197075,0.313786,0.353486,0.224127,0.121282,0.372948,0.048265,-0.010551
5064,1,Twentyfirst Century Mgmt. Services Ltd.,21STCENMGM,2014,0.00,549.7,-123.60,0.463344,-0.170820,0.287611,...,-3.340717,-7.918447,0.380724,-0.187319,0.252789,0.164055,0.074134,0.449927,0.059442,0.048265
5065,1,Twentyfirst Century Mgmt. Services Ltd.,21STCENMGM,2015,18.80,946.1,254.70,-0.099250,0.095392,0.423317,...,-8.461575,-5.079704,-0.104527,0.091112,0.352990,0.331371,0.058602,0.223831,0.109920,0.059442


In [ ]:
reg_name_table_sec1[reg_name_table_sec1.FCF_1 < -1]

,FirmID,Company Name,NSE symbol,Year,CapEx,TA,FCF,FCF_1,FCF_2,Leverage,...,LT(I/TA_1),LT(I/TA_2),LT(FCF_1),LT(FCF_2),LT(Leverage),LT(Revenue),LT(Cash),LT(TQ),LT(TQ_diff_1),LT(TQ_diff_2)
1265,80,Delta Corp Ltd.,DELTACORP,2007,148.8,1601.7,216.7,-1.156459,-0.599238,0.807142,...,-0.701306,-0.520260,NaN,-0.914388,0.591747,0.166357,0.000499,1.408944,NaN,NaN
1200,82,D L F Ltd.,DLF,2006,6466.6,70359.0,-15196.8,-1.221488,-0.932938,0.856598,...,-0.902083,0.752941,NaN,-2.702141,0.618746,0.232141,0.000362,2.091777,NaN,NaN
1696,112,G H C L Ltd.,GHCL,2006,3108.1,13041.0,-3454.8,-1.001610,-0.107852,0.779097,...,0.581347,-1.847924,NaN,-0.114123,0.576106,0.372391,0.008111,0.771279,NaN,NaN


In [ ]:
reg_name_table_sec1[reg_name_table_sec1.FCF_2 < -1]

,FirmID,Company Name,NSE symbol,Year,CapEx,TA,FCF,FCF_1,FCF_2,Leverage,...,LT(I/TA_1),LT(I/TA_2),LT(FCF_1),LT(FCF_2),LT(Leverage),LT(Revenue),LT(Cash),LT(TQ),LT(TQ_diff_1),LT(TQ_diff_2)
916,46,Brightcom Group Ltd.,BCG,2010,91.4,3409.8,96.4,0.010265,-1.119567,0.638425,...,-3.088592,1.115971,0.010212,NaN,0.493735,0.754968,0.000410,1.415993,NaN,NaN
1264,80,Delta Corp Ltd.,DELTACORP,2006,68.0,1189.9,-1.0,0.182116,-1.556685,0.769056,...,-1.945430,-0.215810,0.167306,NaN,0.570446,0.328723,0.000420,3.159045,NaN,NaN
3537,214,New Delhi Television Ltd.,NDTV,2007,425.5,4221.3,-432.5,-0.611305,-1.187549,0.227726,...,-1.333476,-1.836468,-0.944959,NaN,0.205164,0.518810,0.000213,1.839403,NaN,NaN
3808,233,Paramount Communications Ltd.,PARACABLES,2006,86.3,1726.2,-331.7,-0.539451,-1.402850,0.672576,...,-0.847188,0.128319,-0.775336,NaN,0.514365,0.837543,0.001505,0.502668,NaN,NaN
4384,273,Sobha Ltd.,SOBHA,2006,595.2,10020.9,-1247.0,-0.602900,-1.151144,0.872167,...,-2.020326,-2.847986,-0.923567,NaN,0.627097,0.484847,0.000479,2.094726,NaN,NaN
2522,301,Investment Trust Of India Ltd.,THEINVEST,2016,13.1,3941.1,3710.7,0.177032,-2.080155,0.180483,...,-5.351323,-4.901066,0.162996,NaN,0.165923,0.045000,0.196552,1.228700,-1.067907,NaN


In [ ]:
# Removing firms with FCF_1 or FCF_2 lesser than -1. Thus we can apply logistic transformation on the dependent variable
# whilst keeping the panel data balanced.
exclude1 = list(reg_name_table_sec1[reg_name_table_sec1.FCF_1 < -1]['Company Name'])
exclude2 = list(reg_name_table_sec1[reg_name_table_sec1.FCF_2 < -1]['Company Name'])
exclude = list(set(exclude1 + exclude2))
reg_name_table_sec1 = reg_name_table_sec1[~reg_name_table_sec1['Company Name'].isin(exclude)]

In [ ]:
#section1
reg_name_table_sec11 = reg_name_table_sec1[['FirmID','Company Name','NSE symbol','Year','LT(I/TA_1)','Amihud','Turnover','LT(FCF_1)','LT(Leverage)','LT(Revenue)','LT(Cash)','TQ','TQ_diff_1','TQ_diff_2']]
reg_name_table_sec11 = reg_name_table_sec11[reg_name_table_sec11['LT(FCF_1)'].notna()]

reg_name_table_sec12 = reg_name_table_sec1[['FirmID','Company Name','NSE symbol','Year','LT(I/TA_2)','Amihud','Turnover','LT(FCF_2)','LT(Leverage)','LT(Revenue)','LT(Cash)','TQ','TQ_diff_1','TQ_diff_2']]
reg_name_table_sec12 = reg_name_table_sec12[reg_name_table_sec12['LT(FCF_2)'].notna()]

In [ ]:
reg_name_table_sec1.isnull().sum(axis = 0)

FirmID              0
Company Name        0
NSE symbol          0
Year                0
CapEx               0
TA                  0
FCF                 0
FCF_1             319
FCF_2             638
Leverage            0
Revenue             0
Cash                0
TQ                  0
TQ_diff_1         638
TQ_diff_2         957
Amihud              0
Turnover            0
FLR                 0
PR                  0
KZ                319
I_1               319
I_2               638
I/TA_1            319
I/TA_2            638
LT(I/TA_1)          0
LT(I/TA_2)          0
LT(FCF_1)         319
LT(FCF_2)         638
LT(Leverage)        0
LT(Revenue)         0
LT(Cash)            0
LT(TQ)              0
LT(TQ_diff_1)     904
LT(TQ_diff_2)    1213
dtype: int64

In [ ]:
reg_name_table_sec12.head(15)

,FirmID,Company Name,NSE symbol,Year,LT(I/TA_2),Amihud,Turnover,LT(FCF_2),LT(Leverage),LT(Revenue),LT(Cash),TQ,TQ_diff_1,TQ_diff_2
5056,1,Twentyfirst Century Mgmt. Services Ltd.,21STCENMGM,2006,-3.998075,0.079698,0.000535,-0.143348,0.274575,0.477137,0.001480,0.612042,NaN,NaN
5057,1,Twentyfirst Century Mgmt. Services Ltd.,21STCENMGM,2007,-7.053215,0.079698,0.000535,-0.457324,0.285087,0.399731,0.001603,0.292902,NaN,NaN
5058,1,Twentyfirst Century Mgmt. Services Ltd.,21STCENMGM,2008,-6.104315,0.079698,0.000535,0.119073,0.331663,0.262315,0.001113,0.188873,-0.319140,NaN
5059,1,Twentyfirst Century Mgmt. Services Ltd.,21STCENMGM,2009,-3.690069,0.079698,0.000535,0.023189,0.291445,0.068374,0.002189,0.383267,-0.104028,-0.319140
5060,1,Twentyfirst Century Mgmt. Services Ltd.,21STCENMGM,2010,-6.605072,0.079698,0.000535,0.034975,0.364696,0.068873,0.001913,0.351812,0.194393,-0.104028
5061,1,Twentyfirst Century Mgmt. Services Ltd.,21STCENMGM,2011,-5.046178,0.079698,0.000535,0.004090,0.339247,0.285932,0.166053,0.341316,-0.031455,0.194393
5062,1,Twentyfirst Century Mgmt. Services Ltd.,21STCENMGM,2012,-3.576699,0.079698,0.000535,-0.167987,0.375022,0.083031,0.112752,0.390765,-0.010496,-0.031455
5063,1,Twentyfirst Century Mgmt. Services Ltd.,21STCENMGM,2013,-3.576699,0.079698,0.000535,0.313786,0.353486,0.224127,0.121282,0.452009,0.049449,-0.010496
5064,1,Twentyfirst Century Mgmt. Services Ltd.,21STCENMGM,2014,-7.918447,0.079698,0.000535,-0.187319,0.252789,0.164055,0.074134,0.568198,0.061244,0.049449
5065,1,Twentyfirst Century Mgmt. Services Ltd.,21STCENMGM,2015,-5.079704,0.079698,0.000535,0.091112,0.352990,0.331371,0.058602,0.250860,0.116189,0.061244


In [ ]:
reg_name_table_sec11.to_excel("df1.xlsx")
reg_name_table_sec12.to_excel("df2.xlsx")

In [ ]:
reg_name_table_sec1.columns

Index(['FirmID', 'Company Name', 'NSE symbol', 'Year', 'CapEx', 'TA', 'FCF',
       'FCF_1', 'FCF_2', 'Leverage', 'Revenue', 'Cash', 'TQ', 'TQ_diff_1',
       'TQ_diff_2', 'Amihud', 'Turnover', 'FLR', 'PR', 'KZ', 'I_1', 'I_2',
       'I/TA_1', 'I/TA_2', 'LT(I/TA_1)', 'LT(I/TA_2)', 'LT(FCF_1)',
       'LT(FCF_2)', 'LT(Leverage)', 'LT(Revenue)', 'LT(Cash)', 'LT(TQ)',
       'LT(TQ_diff_1)', 'LT(TQ_diff_2)'],
      dtype='object')

In [ ]:
reg_name_table_section1 = reg_name_table_sec1[['FirmID','Company Name','Year','I/TA_1','I/TA_2','LT(I/TA_1)','LT(I/TA_2)','Amihud','Turnover','FCF_1','FCF_2','LT(FCF_1)','LT(FCF_2)','Leverage', 'Revenue', 'Cash','LT(Leverage)', 'LT(Revenue)', 'LT(Cash)','TQ', 'TQ_diff_1',
       'TQ_diff_2','FLR', 'PR', 'KZ']]

In [ ]:
reg_name_table_section1.to_excel("data.xlsx")

In [ ]:
reg_name_table_section2 = reg_name_table_section1
reg_name_table_section2 = reg_name_table_section2.sort_values(by=['Year','Company Name'], ascending=True)
HLvalues = []
HLvaluesKZ = []
for j in range(16):
    for i in range(319):
        if i < 239:
            HLvalues.append(0)
        else:
            HLvalues.append(1)

for k in range(319):
    HLvaluesKZ.append(0)

for l in range(15):
    for m in range(319):
        if m < 239:
            HLvaluesKZ.append(0)
        else:
            HLvaluesKZ.append(1)

reg_name_table_section2 = reg_name_table_section2.sort_values(by=['Year','FLR'], ascending=True)
reg_name_table_section2['High_FLR'] = HLvalues
reg_name_table_section2 = reg_name_table_section2.sort_values(by=['Year','PR'], ascending=True)
reg_name_table_section2['High_PR'] = HLvalues
reg_name_table_section2 = reg_name_table_section2.sort_values(by=['Year','KZ'], ascending=True)
reg_name_table_section2['High_KZ'] = HLvaluesKZ
reg_name_table_section2 = reg_name_table_section2.sort_values(by=['FirmID','Year'], ascending=True)


reg_name_table_section2['Amihud X High_FLR'] = reg_name_table_section2['Amihud']* reg_name_table_section2['High_FLR']
reg_name_table_section2['Amihud X High_PR'] = reg_name_table_section2['Amihud']* reg_name_table_section2['High_PR']
reg_name_table_section2['Amihud X High_KZ'] = reg_name_table_section2['Amihud']* reg_name_table_section2['High_KZ']

reg_name_table_section2['Turnover X High_FLR'] = reg_name_table_section2['Turnover']* reg_name_table_section2['High_FLR']
reg_name_table_section2['Turnover X High_PR'] = reg_name_table_section2['Turnover']* reg_name_table_section2['High_PR']
reg_name_table_section2['Turnover X High_KZ'] = reg_name_table_section2['Turnover']* reg_name_table_section2['High_KZ']

In [ ]:
reg_name_table_section2

,FirmID,Company Name,Year,I/TA_1,I/TA_2,LT(I/TA_1),LT(I/TA_2),Amihud,Turnover,FCF_1,...,KZ,High_FLR,High_PR,High_KZ,Amihud X High_FLR,Amihud X High_PR,Amihud X High_KZ,Turnover X High_FLR,Turnover X High_PR,Turnover X High_KZ
5056,1,Twentyfirst Century Mgmt. Services Ltd.,2006,0.013824,0.018020,-4.267447,-3.998075,7.969760e-02,0.000535,-0.026660,...,NaN,0,0,0,0.0,0.000000e+00,0.000000,0.0,0.000000,0.000000
5057,1,Twentyfirst Century Mgmt. Services Ltd.,2007,0.009009,0.000864,-4.700480,-7.053215,7.969760e-02,0.000535,-0.066765,...,-19.826341,0,0,0,0.0,0.000000e+00,0.000000,0.0,0.000000,0.000000
5058,1,Twentyfirst Century Mgmt. Services Ltd.,2008,0.000557,0.002228,-7.492283,-6.104315,7.969760e-02,0.000535,-0.236670,...,25.831194,0,0,1,0.0,0.000000e+00,0.079698,0.0,0.000000,0.000535
5059,1,Twentyfirst Century Mgmt. Services Ltd.,2009,0.003609,0.024362,-5.620660,-3.690069,7.969760e-02,0.000535,0.204821,...,24.631360,0,0,1,0.0,0.000000e+00,0.079698,0.0,0.000000,0.000535
5060,1,Twentyfirst Century Mgmt. Services Ltd.,2010,0.021289,0.001352,-3.828066,-6.605072,7.969760e-02,0.000535,0.020500,...,2.361443,0,0,0,0.0,0.000000e+00,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971,344,Cadila Healthcare Ltd.,2017,0.067969,0.068494,-2.618313,-2.610060,3.052272e-07,0.001172,-0.008421,...,58.502373,0,1,0,0.0,3.052272e-07,0.000000,0.0,0.001172,0.000000
972,344,Cadila Healthcare Ltd.,2018,0.058091,0.049669,-2.785894,-2.951424,3.455713e-07,0.001477,0.012334,...,52.549984,0,0,0,0.0,0.000000e+00,0.000000,0.0,0.000000,0.000000
973,344,Cadila Healthcare Ltd.,2019,0.037828,0.035732,-3.236149,-3.295334,3.969237e-07,0.001680,0.084827,...,17.736089,0,0,0,0.0,0.000000e+00,0.000000,0.0,0.000000,0.000000
974,344,Cadila Healthcare Ltd.,2020,0.036008,NaN,-3.287335,-0.712072,1.697044e-07,0.004472,0.102838,...,17.127658,0,1,0,0.0,1.697044e-07,0.000000,0.0,0.004472,0.000000


In [ ]:
reg_name_table_section2.to_excel("newfinal.xlsx")